In [84]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_curve
import os

In [89]:
scraped_papers = pd.read_csv("data/scraped_papers_final.csv")
scraped_papers

,Unnamed: 0,DOI,URL,year,title,publisher,text,label
0,0,10.1002/9781119578826.ch15,https://doi.org/10.1002/9781119578826.ch15,2024,['Quantum Dots Solar Cells'],Wiley,Paper #: 7386_1\nTitle: Role of carrier deloca...,0
1,1,10.1002/9781119578826.ch16,https://doi.org/10.1002/9781119578826.ch16,2024,['Singlet Fission for Solar Cells'],Wiley,Paper #: 7742_1\nTitle: Perovskite-Based Tande...,0
2,2,10.1002/9781119578826.ch27,https://doi.org/10.1002/9781119578826.ch27,2024,['Deployment of<scp>Solar</scp>Photovoltaic Sy...,Wiley,Paper #: 9299_1\nTitle: Inhomogeneous Halide A...,1
3,3,10.1002/9781119578826.ch6,https://doi.org/10.1002/9781119578826.ch6,2024,['Tunnel Oxide Passivated Contact (<scp>TOPCon...,Wiley,Paper #: 7949_1\nTitle: Efficiency assessment ...,0
4,5,10.1002/9781119600862.ch3,https://doi.org/10.1002/9781119600862.ch3,2024,['Solar Thermochemical Fuels'],Wiley,Paper #: 7743_1\nTitle: Manufacturing Cost Ana...,0
...,...,...,...,...,...,...,...,...
9873,10259,10.7836/kses.2023.43.5.043,https://doi.org/10.7836/kses.2023.43.5.043,2023,['Performance Evaluation of LSTM Based Solar I...,The Korean Solar Energy Society,Paper #: 16412_1\nTitle: Supporting Informatio...,0
9874,10260,10.7836/kses.2023.43.6.013,https://doi.org/10.7836/kses.2023.43.6.013,2024,['A Methodology for Calculating the Limit Capa...,The Korean Solar Energy Society,Paper #: 16980_1\nTitle: Nanosecond Laser Pass...,0
9875,10261,10.7836/kses.2023.43.6.067,https://doi.org/10.7836/kses.2023.43.6.067,2024,['Prediction Model For Colored BIPV Modules'],The Korean Solar Energy Society,Paper #: 19568_1\nTitle: Improve Perovskite So...,0
9876,10262,10.7836/kses.2023.43.6.087,https://doi.org/10.7836/kses.2023.43.6.087,2024,['Transformer-Based Photovoltaic Power Predict...,The Korean Solar Energy Society,Paper #: 19781_1\nTitle: Electron-hole diffusi...,1


In [90]:
classifier_model = pickle.load(open("models/classifier_svm.pkl", "rb"))

In [91]:
tfidf_vectorizer = TfidfVectorizer(max_features=4000)  # Adjust max_features as needed
vectorized = tfidf_vectorizer.fit_transform(scraped_papers['text']) 

# scraped_papers["label"] = classifier_model.predict(vectorized)
vectorized

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11308277 stored elements and shape (9878, 4000)>

In [95]:
scraped_papers["label"] = classifier_model.predict(vectorized)

In [96]:
y_scores = classifier_model.decision_function(vectorized)

threshold = -0.4

scraped_papers["label"] = (y_scores > threshold).astype(int)

In [98]:
relevant_papers = scraped_papers[scraped_papers["label"] == 1]
relevant_papers.shape[0]

2860

In [99]:
folder_path = "data/relevant_samples"

sampled_df = relevant_papers.sample(n=40, random_state=42)

for i, text in enumerate(sampled_df["text"]):
    file_path = os.path.join(folder_path, f"sample_{i}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

In [100]:
relevant_papers.to_csv("data/relevant_papers.csv", index=False)

In [101]:
relevant_papers["year"].unique()

array([2024, 2023, 2025, 2022, 2017, 2019, 2021, 2020])